In [1]:
import sqlite3
import pandas as pd
db = './sqlite_db_pythonsqlite.db'

conn = sqlite3.connect(db)
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = []
for table in c.fetchall():
    print(table[0])
    tables.append(pd.read_sql(sql= f"SELECT * FROM {table[0]};",con=conn))



Bookings
Facilities
Members


In [2]:
bookings = tables[0]
facilities = tables[1]
members = tables[2]

In [3]:
facilities.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [4]:
bookings.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [5]:
members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


In [6]:
#Q10
answer10 = []

for i in range(len(facilities)):
    fac_bookings = bookings[bookings['facid'] == facilities['facid'][i]]

    revenue = facilities['guestcost'][i]*sum(fac_bookings[fac_bookings['memid'] == 0]['slots'])
    revenue += facilities['membercost'][i]*sum(fac_bookings[fac_bookings['memid'] != 0]['slots'])

    if revenue < 1000:
        answer10.append(facilities['name'][i])


print(answer10)

    
    
    

['Table Tennis', 'Snooker Table', 'Pool Table']


In [7]:
#Q11

answer11 = {
    'member':[],
    'recommender':[]
}

for i in range(len(members)):
    member = members.iloc[i]
    recommender = member['recommendedby']
    answer11['member'].append(member['surname']+','+member['firstname'])
    if len(recommender) > 0:
        recommender_member = members[members['memid'] == int(recommender)]
        answer11['recommender'].append(recommender_member['surname'].iloc[0]+','+ recommender_member['firstname'].iloc[0])
    else:
        answer11['recommender'].append('')

pd.DataFrame(answer11)

,member,recommender
0,"GUEST,GUEST",
1,"Smith,Darren",
2,"Smith,Tracy",
3,"Rownam,Tim",
4,"Joplette,Janice","Smith,Darren"
5,"Butters,Gerald","Smith,Darren"
6,"Tracy,Burton",
7,"Dare,Nancy","Joplette,Janice"
8,"Boothe,Tim","Rownam,Tim"
9,"Stibbons,Ponder","Tracy,Burton"


In [8]:
#Q12

answer12 = {
    'fac_name':[],
    'usage':[]
}
for i in range(len(facilities)):
    fac_bookings = bookings[(bookings['facid'] == facilities['facid'][i]) & (bookings['memid'] > 0)]

    answer12['fac_name'].append(facilities['name'][i])
    answer12['usage'].append(sum(fac_bookings['slots']))

pd.DataFrame(answer12)

,fac_name,usage
0,Tennis Court 1,957
1,Tennis Court 2,882
2,Badminton Court,1086
3,Table Tennis,794
4,Massage Room 1,884
5,Massage Room 2,54
6,Squash Court,418
7,Snooker Table,860
8,Pool Table,856


In [10]:
#Q13

bookings = bookings.merge(right=facilities,how='inner')
bookings['year'] = pd.to_datetime(bookings['starttime']).dt.year
bookings['month'] = pd.to_datetime(bookings['starttime']).dt.month
bookings = bookings[bookings['memid'] > 0]
answer13 = bookings[['month','year','name','slots']].groupby(by=['month','year','name']).agg(sum)

answer13


slots
month year name                  
7     2012 Badminton Court    165
           Massage Room 1     166
           Massage Room 2       8
           Pool Table         110
           Snooker Table      140
           Squash Court        50
           Table Tennis        98
           Tennis Court 1     201
           Tennis Court 2     123
8     2012 Badminton Court    414
           Massage Room 1     316
           Massage Room 2      18
           Pool Table         303
           Snooker Table      316
           Squash Court       184
           Table Tennis       296
           Tennis Court 1     339
           Tennis Court 2     345
9     2012 Badminton Court    507
           Massage Room 1     402
           Massage Room 2      28
           Pool Table         443
           Snooker Table      404
           Squash Court       184
           Table Tennis       400
           Tennis Court 1     417
           Tennis Court 2     414

In [11]:
conn.close()